# Static elastic constants
This Notebook follows the workflow for setting up, running, and processing the elastic_constants_static calculations.

**Quick Notes:**

- All input scripts take key-value pairs where the "key"s correspond to the calculation's input parameters.

- Multiple calculations are prepared by specifying multiple values for the same key (on separate lines).

- The special "buildcombos" key accesses predefined functions for generating lists of input parameter values for certain sets of keys.  See documentation for more details.

__Global workflow details:__

This Notebook requires calculation_crystal_space_group records and the "unique_crystals.csv" file, both of which are generated by the "3. Crystal relaxation" Notebook.

The calculation_elastic_constants_static records generated are used as inputs for dislocation creation Notebooks.

**Library imports**

In [1]:
# Standard Python libraries
from __future__ import (absolute_import, print_function,
                        division, unicode_literals)
import os
from datetime import date
from math import floor
from copy import deepcopy

# http://www.numpy.org/
import numpy as np

# https://pandas.pydata.org/
import pandas as pd

# https://bokeh.pydata.org/
import bokeh
import bokeh.plotting
import bokeh.resources
import bokeh.io
import bokeh.models

from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy
print('iprPy version', iprPy.__version__)

iprPy version 0.8.3


## 0. Access database 

### Load database

In [2]:
database = iprPy.load_database('potential_testing')

### Load unique crystals

In [3]:
unique_crystals = pd.read_csv('unique_crystals.csv')
print(len(unique_crystals))

15


### Limit unique crystals to include (optional) 

In [4]:
#unique_crystals = unique_crystals[unique_crystals.potential_LAMMPS_id == '2012--Park-H--Mo--LAMMPS--ipr1']
#unique_crystals = unique_crystals[unique_crystals.composition == '']
#unique_crystals = unique_crystals[unique_crystals.prototype == '']
print(len(unique_crystals))

15


## 1. Run elastic_constants_static calculations

This calculation evaluates the elastic constants for a crystal using small strain deformations.

In [5]:
calculation = iprPy.load_calculation('elastic_constants_static')
run_directory = iprPy.load_run_directory('potential_testing_1')

### Show calculation's allowed keys

These are the keys that the input script will accept.

In [6]:
print(calculation.allkeys)

['lammps_command', 'mpi_command', 'length_unit', 'pressure_unit', 'energy_unit', 'force_unit', 'potential_file', 'potential_content', 'potential_dir', 'load_file', 'load_content', 'load_style', 'family', 'load_options', 'symbols', 'box_parameters', 'a_uvw', 'b_uvw', 'c_uvw', 'atomshift', 'sizemults', 'strainrange', 'energytolerance', 'forcetolerance', 'maxiterations', 'maxevaluations', 'maxatommotion']


### Write input script

In [7]:
input_script = """
# Commands and executables
lammps_command              lmp_mpi
mpi_command                 

# Build load information from crystal_space_group results
buildcombos                 atomicparent load_file parent
parent_record               calculation_crystal_space_group
parent_load_key             unit-cell-atomic-system

# System manipulations
a_uvw                      
b_uvw                      
c_uvw                 
atomshift                   
sizemults                   5 5 5

# Units that input/output values are in
length_unit                 
pressure_unit               
energy_unit                 
force_unit                  

# Run parameters
energytolerance             
forcetolerance              
maxiterations               
maxevaluations              
maxatommotion               
strainrange                  1e-5
strainrange                  1e-6
strainrange                  1e-7
strainrange                  1e-8
"""

with open('input_script.in', 'w') as f:
    f.write(input_script)

### Parse input script

In [8]:
with open('input_script.in') as f:
    input_dict = iprPy.input.parse(f, singularkeys=calculation.singularkeys)

### Add parent_name list using unique crystals

In [9]:
input_dict['parent_name'] = unique_crystals.calc_key.tolist()

### Prepare calculations

In [10]:
database.prepare(run_directory, calculation, **input_dict)

In [11]:
database.check_records(calculation.record_style)

In database style local at C:\Users\lmh1\Documents\calculations\ipr\potential_testing :
- 60 of style calculation_elastic_constants_static
 - 0 are complete
 - 60 still to run
 - 0 issued errors


### Run calculations

In [12]:
database.runner(run_directory)

Runner started with pid 9940
No simulations left to run


In [13]:
results_df = database.get_records_df(style=calculation.record_style)
error_df = results_df[results_df.status=='error']
print(len(error_df), 'calculations issued errors:')
errors = []
for error in error_df.error:
    lines = error.splitlines()
    err = ''
    for i in range(len(lines)-1, -1, -1):
        if 'Error:' in lines[i]:
            err = '\n'.join(lines[i:-1])
            break
        if i == 0:
            err = error
    errors.append(err)
for error in np.unique(errors):
    print(error)

4 calculations issued errors:
ValueError: Invalid LAMMPS input: 
Last command: read_restart initial.restart


## 2. Calculation analysis

### Retrieve finished calculation results

In [14]:
records = database.get_records_df(style='calculation_elastic_constants_static', full=True, flat=False, status='finished')
pot_records = database.get_records_df(style='potential_users_LAMMPS')

### Identify compositions

In [15]:
iprPy.analysis.assign_composition(records, database)

In [16]:
# Settings
outputpath = 'C:/Users/lmh1/Documents/potential_testing_results'
#savecolumns = ['calc_key', 'composition', 'prototype', 'family', 'a', 
#               'strainrange', '', 'gamma_fs']

### Save raw data

In [17]:
records

,C,LAMMPS_version,error,family,iprPy_version,key,load_file,load_options,load_style,potential_LAMMPS_id,...,potential_id,potential_key,raw_cij_negative,raw_cij_positive,script,sizemults,status,strainrange,symbols,composition
0,[[0.72412699 0.34635333 0.28219474 0. ...,26 Jan 2017-ICMS,NaN,oqmd-1215215,0.8.3,03717544-ca87-413b-bdcf-229bdf2021d5,b0ffec08-bb68-4ca4-9608-d5d259985078.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.7241291287979883, 0.34635402080044725, 0.2...","[[0.7241248548108673, 0.34635264837430807, 0.2...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-06,[Al],Al
1,[[0.77528073 0.4289165 0.4289165 0. ...,26 Jan 2017-ICMS,NaN,oqmd-1214770,0.8.3,03e6ea22-5e0b-4ff3-8953-cc915c72e614,2784b307-7960-457c-9901-d495ce493e92.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.7678827342170512, 0.4192493822639433, 0.41...","[[0.7826787300510274, 0.4385836101416474, 0.43...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-06,[Al],Al
3,[[ 0.85079137 0.15530357 0.08940042 0. ...,26 Jan 2017-ICMS,NaN,A5--beta-Sn,0.8.3,07465678-705f-4bea-b5e6-ac2b9452571f,42610089-f823-4d79-9402-65161599ad0c.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.8507931361375239, 0.15530420666069905, 0.0...","[[0.8507895941821524, 0.15530292657758424, 0.0...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,[Al],Al
4,[[ 0.39073819 0.38305496 -0.00843477 0. ...,26 Jan 2017-ICMS,NaN,oqmd-1214681,0.8.3,0af62a50-e7c9-4104-a4d5-d9ea1d4f56d8,4c4cccea-2d40-4635-b171-c18048f2679f.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.3904249086320033, 0.4151132743622499, -0.0...","[[0.39105146566742466, 0.41465613685384706, -0...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-06,[Al],Al
5,[[0.71025763 0.38419452 0.38419452 0. ...,26 Jan 2017-ICMS,NaN,A1--Cu--fcc,0.8.3,156a9fb5-cb6b-4506-9e24-862c185fe156,7c0c7d7e-3b04-4609-8aaa-97965e15d509.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.7102577156739747, 0.38419450316120624, 0.3...","[[0.7102575502227653, 0.38419453518975744, 0.3...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,[Al],Al
6,[[ 7.63043321e-01 4.37944627e-01 4.37944626e...,26 Jan 2017-ICMS,NaN,oqmd-1214770,0.8.3,1929e8a1-5aa5-41e1-90ba-123d46b666aa,649d7093-d649-4df3-958c-a7ed224053d9.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.766421428708905, 0.42889303152830044, 0.42...","[[0.7596652139372356, 0.44699622229549607, 0.4...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-08,[Al],Al
7,[[0.65662387 0.3923673 0.3923673 0. ...,26 Jan 2017-ICMS,NaN,oqmd-1214859,0.8.3,21d2946e-7014-436a-97dc-789faba091cb,7374cb15-6cd9-499f-847d-07b9f23c9678.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.593510813834278, 0.32925363129400304, 0.32...","[[0.7197369354867458, 0.455480965671053, 0.455...",calc_elastic_constants_static,"[[0, 5], [0, 5], [0, 5]]",finished,1.000000e-07,[Al],Al
8,[[ 0.39067656 0.38308282 -0.0084637 0. ...,26 Jan 2017-ICMS,NaN,oqmd-1214681,0.8.3,30cb1e5f-0001-426f-8804-d5a591877725,4c4cccea-2d40-4635-b171-c18048f2679f.json,key unit-cell-atomic-system,system_model,1999--Mishin-Y--Al--LAMMPS--testing1,...,1999--Mishin-Y--Al,eb33d631-9ac6-4b51-8642-7a92462b96a3,"[[0.38835176699447693, 0.4

In [18]:
data = []
for record in records.itertuples():
    crystal = unique_crystals[unique_crystals.calc_key == record.load_file.split('.')[0]].iloc[0]
    
    dat = {}
    dat['calc_key'] = record.key
    dat['potential_LAMMPS_key'] = crystal.potential_LAMMPS_key
    dat['potential_LAMMPS_id'] = crystal.potential_LAMMPS_id
    dat['potential_key'] = crystal.potential_key
    dat['potential_id'] = crystal.potential_id
    dat['prototype'] = crystal.prototype
    dat['family'] = crystal.family
    dat['composition'] = record.composition
    dat['strainrange'] = record.strainrange
    dat['a'] = crystal.a
    dat['b'] = crystal.b
    dat['c'] = crystal.c
    dat['alpha'] = crystal.alpha
    dat['beta'] = crystal.beta
    dat['gamma'] = crystal.gamma

    dat['straindirection'] = 'positive'
    cij = uc.get_in_units(record.raw_cij_positive, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

    dat['straindirection'] = 'negative'
    cij = uc.get_in_units(record.raw_cij_negative, 'GPa')
    dat['C11'] = '%.3f' %cij[0,0]
    dat['C12'] = '%.3f' %cij[0,1]
    dat['C13'] = '%.3f' %cij[0,2]
    dat['C14'] = '%.3f' %cij[0,3]
    dat['C15'] = '%.3f' %cij[0,4]
    dat['C16'] = '%.3f' %cij[0,5]
    dat['C21'] = '%.3f' %cij[1,0]
    dat['C22'] = '%.3f' %cij[1,1]
    dat['C23'] = '%.3f' %cij[1,2]
    dat['C24'] = '%.3f' %cij[1,3]
    dat['C25'] = '%.3f' %cij[1,4]
    dat['C26'] = '%.3f' %cij[1,5]
    dat['C31'] = '%.3f' %cij[2,0]
    dat['C32'] = '%.3f' %cij[2,1]
    dat['C33'] = '%.3f' %cij[2,2]
    dat['C34'] = '%.3f' %cij[2,3]
    dat['C35'] = '%.3f' %cij[2,4]
    dat['C36'] = '%.3f' %cij[2,5]
    dat['C41'] = '%.3f' %cij[3,0]
    dat['C42'] = '%.3f' %cij[3,1]
    dat['C43'] = '%.3f' %cij[3,2]
    dat['C44'] = '%.3f' %cij[3,3]
    dat['C45'] = '%.3f' %cij[3,4]
    dat['C46'] = '%.3f' %cij[3,5]
    dat['C51'] = '%.3f' %cij[4,0]
    dat['C52'] = '%.3f' %cij[4,1]
    dat['C53'] = '%.3f' %cij[4,2]
    dat['C54'] = '%.3f' %cij[4,3]
    dat['C55'] = '%.3f' %cij[4,4]
    dat['C56'] = '%.3f' %cij[4,5]
    dat['C61'] = '%.3f' %cij[5,0]
    dat['C62'] = '%.3f' %cij[5,1]
    dat['C63'] = '%.3f' %cij[5,2]
    dat['C64'] = '%.3f' %cij[5,3]
    dat['C65'] = '%.3f' %cij[5,4]
    dat['C66'] = '%.3f' %cij[5,5]
    data.append(deepcopy(dat))

columns = ['calc_key', 'potential_LAMMPS_key', 'potential_LAMMPS_id', 'potential_key', 'potential_id',
           'composition', 'prototype', 'family', 
           'a', 'b', 'c', 'alpha', 'beta', 'gamma',
           'strainrange', 'straindirection', 
           'C11', 'C12', 'C13', 'C14', 'C15', 'C16',
           'C21', 'C22', 'C23', 'C24', 'C25', 'C26',
           'C31', 'C32', 'C33', 'C34', 'C35', 'C36',
           'C41', 'C42', 'C43', 'C44', 'C45', 'C46',
           'C51', 'C52', 'C53', 'C54', 'C55', 'C56',
           'C61', 'C62', 'C63', 'C64', 'C65', 'C66']

data = pd.DataFrame(data, columns=columns).sort_values(['potential_LAMMPS_id', 'composition', 'prototype', 'strainrange', 'straindirection'])

In [19]:
data.to_csv('elastic-constants.csv')

### Save raw crystal-specific data

In [20]:
for implememtation_key in np.unique(data.potential_LAMMPS_key):
    imp_results = data[data.potential_LAMMPS_key == implememtation_key]
    potential = imp_results.iloc[0].potential_id
    implementation = imp_results.iloc[0].potential_LAMMPS_id
    
    contentpath = os.path.join(outputpath, potential, implementation)
    if not os.path.isdir(contentpath):
        os.makedirs(contentpath)
    
    for composition in np.unique(imp_results.composition):
        comp_results = imp_results[imp_results.composition == composition].sort_values(['family', 'a', 'strainrange', 'straindirection'])
        fstem = 'elastic.' + composition
        
        comp_results.to_csv(os.path.join(contentpath, fstem + '.csv'), index=False)


### Add info to PotentialProperties records

This is for generating XML records that the Interatomic Potential Repository uses to automatically build webcontent (done elsewhere).

In [21]:
# Add calculation data to PotentialProperties records
for implememtation_key in np.unique(data.potential_LAMMPS_key):
    imp_records = data[data.potential_LAMMPS_key == implememtation_key].sort_values('family')
    pot_record = pot_records[pot_records.key == implememtation_key].iloc[0]
    implementation_id = pot_record.id
    potential_id = pot_record.pot_id
    potential_key = pot_record.pot_key
    
    record_name = 'properties.' + implementation_id
    try:
        record = database.get_record(name=record_name, style='PotentialProperties')
    except:
        new = True
        content = DM()
        content['per-potential-properties'] = DM()
        content['per-potential-properties']['potential'] = DM()
        content['per-potential-properties']['potential']['key'] = potential_key
        content['per-potential-properties']['potential']['id'] = potential_id
        content['per-potential-properties']['implementation'] = DM()
        content['per-potential-properties']['implementation']['key'] = implememtation_key
        content['per-potential-properties']['implementation']['id'] = implementation_id
    else:
        content = DM(record.content)
        new = False
    
    content['per-potential-properties']['elastic-constants'] = ec_model = DM()
    
    for composition in np.unique(imp_records.composition):
        comp_records = imp_records[imp_records.composition == composition]
        comp_model = DM()
        comp_model['composition'] = composition
        
        for prototype in np.unique(comp_records.prototype):
            proto_records = comp_records[comp_records.prototype == prototype]
            proto_model = DM()
            proto_model['prototype'] = prototype
            
            for alat in np.unique(proto_records.a):
                alat_records = proto_records[proto_records.a == alat]
                alat_model = DM()
                alat_model['a'] = alat
                
                for series in alat_records.sort_values(['strainrange', 'straindirection']).itertuples():
                    measurement = DM()
                    if series.straindirection == 'positive':
                        measurement['strain'] = '+%s' %series.strainrange
                    elif series.straindirection == 'negative':
                        measurement['strain'] = '-%s' %series.strainrange
                    else:
                        raise ValueError('Unknown straindirection')
                    measurement['C11'] = series.C11
                    measurement['C12'] = series.C12
                    measurement['C13'] = series.C13
                    measurement['C14'] = series.C14
                    measurement['C15'] = series.C15
                    measurement['C16'] = series.C16
                    measurement['C21'] = series.C21
                    measurement['C22'] = series.C22
                    measurement['C23'] = series.C23
                    measurement['C24'] = series.C24
                    measurement['C25'] = series.C25
                    measurement['C26'] = series.C26
                    measurement['C31'] = series.C31
                    measurement['C32'] = series.C32
                    measurement['C33'] = series.C33
                    measurement['C34'] = series.C34
                    measurement['C35'] = series.C35
                    measurement['C36'] = series.C36
                    measurement['C41'] = series.C41
                    measurement['C42'] = series.C42
                    measurement['C43'] = series.C43
                    measurement['C44'] = series.C44
                    measurement['C45'] = series.C45
                    measurement['C46'] = series.C46
                    measurement['C51'] = series.C51
                    measurement['C52'] = series.C52
                    measurement['C53'] = series.C53
                    measurement['C54'] = series.C54
                    measurement['C55'] = series.C55
                    measurement['C56'] = series.C56
                    measurement['C61'] = series.C61
                    measurement['C62'] = series.C62
                    measurement['C63'] = series.C63
                    measurement['C64'] = series.C64
                    measurement['C65'] = series.C65
                    measurement['C66'] = series.C66
                    
                    alat_model.append('measurement', measurement)
                proto_model.append('alats', alat_model)
            comp_model.append('prototypes', proto_model)
        ec_model.append('compositions', comp_model)
    
    if new:
        database.add_record(name=record_name, style='PotentialProperties', content=content.xml())
    else:
        database.update_record(record=record, content=content.xml())